## Desafio 2 - Instance-Level Nested Cross-Validation Without Similarity Bias

In [3]:
# Artifício para calcular tempo total do notebook Jupyter
from datetime import datetime
start_time = datetime.now()

import pandas as pd
import numpy as np
import seaborn as sns
import logging
import warnings
import sys
from math import ceil
from matplotlib import pyplot as plt
from time import time
from pathlib import Path
from tsfresh.feature_extraction import extract_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import ParameterGrid, KFold

logging.getLogger('tsfresh').setLevel(logging.ERROR)
warnings.simplefilter(action='ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'png'

data_path = Path('./', 'data')
random_state = 1
np.random.seed(random_state)
events_names = {0: 'Normal',
                1: 'Aumento Abrupto de BSW',
                2: 'Fechamento Espúrio de DHSV',
                3: 'Intermitência Severa',
                4: 'Instabilidade de Fluxo',
                5: 'Perda Rápida de Produtividade',
                6: 'Restrição Rápida em CKP',
                7: 'Incrustação em CKP',
                8: 'Hidrato em Linha de Produção'
               }
vars = ['P-PDG',
        'P-TPT',
        'T-TPT',
        'P-MON-CKP',
        'T-JUS-CKP',
        'P-JUS-CKGL',
        'T-JUS-CKGL',
        'QGL']
columns = ['timestamp'] + vars + ['class']
normal_class_code = 0
abnormal_classes_codes = [1, 2, 5, 6, 7, 8]
sample_size = 3*60              # Nas observações = segundos
min_normal_period_size = 20*60  # Nas observações = segundos
max_samples_per_period = 15     # limitação por 'segurança'
df_fc_p = MinimalFCParameters() # Ver documentação da biblioteca tsfresh
df_fc_p.pop('sum_values')       # Remove feature inapropriada
df_fc_p.pop('length')           # Remove feature inapropriada
max_nan_percent = 0.1           # Para seleção de variáveis úteis
std_vars_min = 0.01             # Para seleção de variáveis úteis
disable_progressbar = True      # Para menos saídas no notebook
n_rounds = 3                    # *** NOVO: 3 rounds de repetição ***
n_folds = 10                    # 10 folds por round
min_instances_for_cv = 10       # Mínimo de instâncias para fazer CV

def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    """Gerador de lista contendo número da classe e caminho do arquivo de acordo com a fonte da instância."""
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

def load_instance(instance_path):
    """Função que carrega cada instância individualmente"""
    try:
        well, instance_id = instance_path.stem.split('_')
        df = pd.read_csv(instance_path, sep=',', header=0)
        assert (df.columns == columns).all(), \
            f'Colunas inválidas no arquivo {str(instance_path)}: {str(df.columns.tolist())}'
        return df
    except Exception as e:
        raise Exception(f'Erro ao ler arquivo {instance_path}: {e}')

def extract_all_samples(df, class_code):
    """Extrai todas as amostras disponíveis de uma instância (normal e anormal)"""
    ols = list(df['class'])
    set_ols = set()
    for ol in ols:
        if ol in set_ols or np.isnan(ol):
            continue
        set_ols.add(int(ol))

    df_vars = df.drop('class', axis=1).fillna(0)
    df_samples = pd.DataFrame()
    y = []
    sample_id = 0

    # Extrai TODAS as amostras do período normal
    f_idx = ols.index(normal_class_code)
    l_idx = len(ols)-1-ols[::-1].index(normal_class_code)
    max_samples_normal = l_idx-f_idx+1-sample_size

    if max_samples_normal > 0:
        num_normal_samples = min(max_samples_per_period, max_samples_normal)
        if num_normal_samples == max_samples_normal:
            step = 1
        else:
            step = (max_samples_normal-1) // (max_samples_per_period-1)
        step = min(sample_size, step)

        for idx in range(num_normal_samples):
            f_idx_c = f_idx + idx*step
            l_idx_c = f_idx_c + sample_size
            df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
            df_sample.insert(loc=0, column='id', value=sample_id)
            df_samples = pd.concat([df_samples, df_sample], ignore_index=True)
            y.append(normal_class_code)
            sample_id += 1

    # Extrai amostras do período transitório (se existir)
    transient_code = class_code + 100
    if transient_code in set_ols:
        f_idx = ols.index(transient_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(transient_code)
        max_transient_samples = l_idx-f_idx+1-sample_size

        if max_transient_samples > 0:
            num_transient_samples = min(max_samples_per_period, max_transient_samples)
            if num_transient_samples == max_transient_samples:
                step = 1
            else:
                step = (max_transient_samples-1) // (max_samples_per_period-1)
            step = min(np.inf, step)

            for idx in range(num_transient_samples):
                f_idx_c = f_idx + int(idx*step)
                l_idx_c = f_idx_c + sample_size
                df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
                df_sample.insert(loc=0, column='id', value=sample_id)
                df_samples = pd.concat([df_samples, df_sample], ignore_index=True)
                y.append(transient_code)
                sample_id += 1

    # Extrai amostras do período em regime (se existir)
    if class_code in set_ols:
        f_idx = ols.index(class_code)
        if f_idx-(sample_size-1) > 0:
            f_idx = f_idx-(sample_size-1)
        else:
            f_idx = 0
        l_idx = len(ols)-1-ols[::-1].index(class_code)
        if l_idx+(sample_size-1) < len(ols)-1:
            l_idx = l_idx+(sample_size-1)
        else:
            l_idx = len(ols)-1
        max_in_regime_samples = l_idx-f_idx+1-sample_size

        if max_in_regime_samples > 0:
            num_in_regime_samples = min(max_samples_per_period, max_in_regime_samples)
            if num_in_regime_samples == max_in_regime_samples:
                step = 1
            else:
                step = (max_in_regime_samples-1) // (max_samples_per_period-1)
            step = min(sample_size, step)

            for idx in range(num_in_regime_samples):
                f_idx_c = f_idx + idx*step
                l_idx_c = f_idx_c + sample_size
                df_sample = df_vars.iloc[f_idx_c:l_idx_c, :].copy()
                df_sample.insert(loc=0, column='id', value=sample_id)
                df_samples = pd.concat([df_samples, df_sample], ignore_index=True)
                y.append(class_code)
                sample_id += 1
    return df_samples, np.array(y)

def get_hyperparameter_grids():
    """Define as grades de hiperparâmetros para cada classificador"""
    param_grids = {
        'Local Outlier Factor': {
            'n_neighbors': [5, 10, 15, 20],
            'contamination': ["auto", 0.01, 0.05, 0.10],
            'algorithm': ['auto']
        },
        'Floresta de Isolamento': {
            'n_estimators': [50, 100, 150, 200],
            'max_samples': ["auto", 0.5, 0.75, 1.0],
            'contamination': ["auto", 0, 0.05, 0.1],
            'max_features': [1.0],
            'bootstrap': [True, False]
        },
        'Envelope Eliptico MCD': {
            'contamination': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.5],
            'support_fraction': [0.95, 0.975, 0.99]
        },
        'One Class SVM': {
            'kernel': ["linear","rbf","poly","sigmoid"],
            'gamma': ["auto", "scale", 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 5, 10],
            'nu': [0.0001, 0.001, 0.01, 0.1, 0.5, 1]
        }
    }
    return param_grids

def create_classifier(clf_name, params):
    """Cria um classificador com os parâmetros especificados"""
    if clf_name == 'Local Outlier Factor':
        return LocalOutlierFactor(
            n_neighbors=params['n_neighbors'],
            contamination=params['contamination'],
            algorithm='auto',
            metric='euclidean',
            novelty=True
        )
    elif clf_name == 'Floresta de Isolamento':
        return IsolationForest(
            n_estimators=params['n_estimators'],
            max_samples=params['max_samples'],
            contamination=params['contamination'],
            max_features=1.0,
            bootstrap=False,
            random_state=random_state,
            verbose=0
        )
    elif clf_name == 'Envelope Eliptico MCD':
        return EllipticEnvelope(
            contamination=params['contamination'],
            support_fraction=params['support_fraction'],
            assume_centered=False,
            random_state=random_state
        )
    elif clf_name == 'One Class SVM':
        return OneClassSVM(
            kernel=params['kernel'],
            gamma=params['gamma'],
            nu=params['nu']
        )

def evaluate_classifier(clf, X_train, y_train, X_val, y_val):
    """Treina e avalia um classificador"""
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    p, r, f1, _ = precision_recall_fscore_support(y_val, y_pred, average='binary', pos_label=1)
    return f1, p, r

def align_features(X_list, feature_union=None):
    """Alinha features de diferentes DataFrames para ter as mesmas colunas"""
    if feature_union is None:
        feature_union = set()
        for X in X_list:
            feature_union.update(X.columns)
        feature_union = sorted(list(feature_union))

    X_aligned = []
    for X in X_list:
        X_new = pd.DataFrame(0, index=X.index, columns=feature_union)
        for col in X.columns:
            if col in feature_union:
                X_new[col] = X[col].values
        X_aligned.append(X_new)

    return X_aligned, feature_union

# Para cada tipo de anomalia
all_results = []

for anomaly_class in abnormal_classes_codes:
    print(f"\n{'='*80}")
    print(f"Processando anomalia: {events_names[anomaly_class]} (código {anomaly_class})")
    print(f"{'='*80}\n")

    # Seleciona instâncias deste tipo de anomalia
    instances_data = []
    real_instances = pd.DataFrame(
        class_and_file_generator(data_path, real=True, simulated=False, drawn=False),
        columns=['class_code', 'instance_path']
    )

    anomaly_instances = real_instances[real_instances['class_code'] == anomaly_class].reset_index(drop=True)

    print(f"Total de instâncias encontradas: {len(anomaly_instances)}")

    # Carrega e processa cada instância
    ignored_instances = 0
    for i, row in anomaly_instances.iterrows():
        class_code, instance_path = row
        print(f"  Carregando instância {i+1}/{len(anomaly_instances)}: {instance_path.name}")

        df = load_instance(instance_path)

        normal_period_size = (df['class'] == float(normal_class_code)).sum()
        if normal_period_size < min_normal_period_size:
            ignored_instances += 1
            print(f"    -> Ignorada (período normal insuficiente: {normal_period_size})")
            continue

        df_samples, y_samples = extract_all_samples(df, class_code)
        y_binary = np.ones(len(y_samples))
        y_binary[y_samples != normal_class_code] = -1

        # Processa features
        good_vars = np.isnan(df_samples[vars]).mean(0) <= max_nan_percent
        std_vars = np.nanstd(df_samples[vars], 0)
        good_vars &= (std_vars > std_vars_min)
        good_vars = list(good_vars.index[good_vars])
        bad_vars = list(set(vars) - set(good_vars))
        df_samples.drop(columns=bad_vars, inplace=True, errors='ignore')

        scaler = preprocessing.StandardScaler()
        df_samples[good_vars] = scaler.fit_transform(df_samples[good_vars]).astype('float32')

        X = extract_features(
            df_samples,
            column_id='id',
            column_sort='timestamp',
            default_fc_parameters=df_fc_p,
            impute_function=impute,
            n_jobs=0,
            disable_progressbar=disable_progressbar
        )
        X = X.reset_index(drop=True)

        instances_data.append({
            'X': X,
            'y': y_binary,
            'instance_path': instance_path
        })

    print(f"\nInstâncias utilizadas: {len(instances_data)}")
    print(f"Instâncias ignoradas: {ignored_instances}")

    if len(instances_data) < min_instances_for_cv:
        print(f"AVISO: Número insuficiente de instâncias ({len(instances_data)}) para CV robusta")
        print(f"Recomendado mínimo: {min_instances_for_cv}. Pulando esta anomalia.\n")
        continue

    n_folds_atual = min(n_folds, len(instances_data))
    print(f"Usando {n_rounds} rounds com {n_folds_atual} folds cada = {n_rounds * n_folds_atual} experimentos totais")

    # Alinha features
    print("Alinhando features entre instâncias...")
    all_X = [inst['X'] for inst in instances_data]
    all_X_aligned, feature_union = align_features(all_X)

    for i in range(len(instances_data)):
        instances_data[i]['X'] = all_X_aligned[i]

    print(f"Total de features após alinhamento: {len(feature_union)}")

    # *** LOOP DE ROUNDS: Repete o processo 3 vezes ***
    for round_num in range(1, n_rounds + 1):
        print(f"\n{'#'*80}")
        print(f"# ROUND {round_num}/{n_rounds}")
        print(f"{'#'*80}\n")

        # Cria KFold com seed diferente para cada round
        round_seed = random_state + round_num * 100
        kf = KFold(n_splits=n_folds_atual, shuffle=True, random_state=round_seed)
        indices = np.arange(len(instances_data))

        # Para cada fold dentro do round
        fold_results = []

        for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(indices)):
            print(f"\n{'-'*80}")
            print(f"ROUND {round_num}/{n_rounds} - FOLD {fold_idx + 1}/{n_folds_atual}")
            print(f"{'-'*80}")

            test_instances = [instances_data[i] for i in test_idx]
            train_val_instances = [instances_data[i] for i in train_val_idx]

            # Divide train_val em treino e validação
            if len(train_val_instances) < 3:
                print("  AVISO: Poucas instâncias, usando hold-one-out")
                train_instances = train_val_instances[:-1]
                val_instances = [train_val_instances[-1]]
            else:
                n_splits_interno = min(3, len(train_val_instances))
                kf_interno = KFold(n_splits=n_splits_interno, shuffle=True, random_state=round_seed)
                train_idx_interno, val_idx_interno = next(kf_interno.split(range(len(train_val_instances))))

                train_instances = [train_val_instances[i] for i in train_idx_interno]
                val_instances = [train_val_instances[i] for i in val_idx_interno]

            # Prepara dados de treino (apenas normais)
            X_train_list = []
            y_train_list = []
            for inst in train_instances:
                normal_mask = inst['y'] == 1
                X_train_list.append(inst['X'][normal_mask])
                y_train_list.append(inst['y'][normal_mask])

            X_train = pd.concat(X_train_list, ignore_index=True)
            y_train = np.concatenate(y_train_list)

            # Prepara dados de validação
            X_val_list = []
            y_val_list = []
            for inst in val_instances:
                X_val_list.append(inst['X'])
                y_val_list.append(inst['y'])

            X_val = pd.concat(X_val_list, ignore_index=True)
            y_val = np.concatenate(y_val_list)

            print(f"  Treino: {len(X_train)} amostras (normais)")
            print(f"  Validação: {len(X_val)} amostras ({(y_val==1).sum()} normais, {(y_val==-1).sum()} anomalias)")

            # Para cada classificador
            param_grids = get_hyperparameter_grids()

            for clf_name in param_grids.keys():
                print(f"\n  Classificador: {clf_name}")

                best_f1 = -1
                best_params = None
                best_scores = None

                grid = list(ParameterGrid(param_grids[clf_name]))
                print(f"    Testando {len(grid)} combinações de hiperparâmetros...")

                for params in grid:
                    try:
                        clf = create_classifier(clf_name, params)
                        f1, p, r = evaluate_classifier(clf, X_train, y_train, X_val, y_val)

                        if f1 > best_f1:
                            best_f1 = f1
                            best_params = params
                            best_scores = (f1, p, r)
                    except Exception as e:
                        continue

                if best_params is None:
                    print(f"    AVISO: Nenhuma combinação funcionou para {clf_name}")
                    continue

                print(f"    Melhor F1 na validação: {best_f1:.4f}")
                print(f"    Melhores hiperparâmetros: {best_params}")

                # Retreina com treino + validação
                X_train_val_list = []
                y_train_val_list = []
                for inst in train_val_instances:
                    normal_mask = inst['y'] == 1
                    X_train_val_list.append(inst['X'][normal_mask])
                    y_train_val_list.append(inst['y'][normal_mask])

                X_train_val = pd.concat(X_train_val_list, ignore_index=True)
                y_train_val = np.concatenate(y_train_val_list)

                # Testa no fold de teste
                X_test_list = []
                y_test_list = []
                for inst in test_instances:
                    X_test_list.append(inst['X'])
                    y_test_list.append(inst['y'])

                X_test = pd.concat(X_test_list, ignore_index=True)
                y_test = np.concatenate(y_test_list)

                print(f"    Retreinando com {len(X_train_val)} amostras normais...")
                print(f"    Testando com {len(X_test)} amostras ({(y_test==1).sum()} normais, {(y_test==-1).sum()} anomalias)")

                try:
                    t0 = time()
                    final_clf = create_classifier(clf_name, best_params)
                    final_clf.fit(X_train_val, y_train_val)
                    t_train = time() - t0

                    t0 = time()
                    y_pred = final_clf.predict(X_test)
                    t_test = time() - t0

                    p, r, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', pos_label=1)

                    print(f"    Resultados no fold de teste - F1: {f1:.4f}, Precisão: {p:.4f}, Revocação: {r:.4f}")

                    fold_results.append({
                        'ANOMALIA': events_names[anomaly_class],
                        'CODIGO_ANOMALIA': anomaly_class,
                        'ROUND': round_num,  # *** NOVO: Identifica o round ***
                        'FOLD': fold_idx + 1,
                        'CLASSIFICADOR': clf_name,
                        'HIPERPARAMETROS': str(best_params),
                        'F1_VALIDACAO': best_f1,
                        'PRECISAO': p,
                        'REVOGACAO': r,
                        'F1': f1,
                        'TREINAMENTO [s]': t_train,
                        'TESTE [s]': t_test
                    })
                except Exception as e:
                    print(f"    Erro ao testar modelo final: {e}")
                    continue

        # Adiciona resultados deste round aos resultados gerais
        all_results.extend(fold_results)

    # Reporta resultados agregados para esta anomalia (todos os rounds)
    if len(all_results) > 0:
        df_anomaly_results = pd.DataFrame([r for r in all_results if r['CODIGO_ANOMALIA'] == anomaly_class])
        if len(df_anomaly_results) > 0:
            print(f"\n{'='*80}")
            print(f"RESUMO FINAL - {events_names[anomaly_class]} ({n_rounds} rounds x {n_folds_atual} folds)")
            print(f"{'='*80}")
            print("\nMédias por classificador (todos os rounds):")
            summary = df_anomaly_results.groupby('CLASSIFICADOR')[['PRECISAO', 'REVOGACAO', 'F1']].mean()
            print(summary.sort_values('F1', ascending=False))
            print("\nDesvios padrão:")
            summary_std = df_anomaly_results.groupby('CLASSIFICADOR')[['PRECISAO', 'REVOGACAO', 'F1']].std()
            print(summary_std)

# Salva resultados finais
print(f"\n{'='*80}")
print(f"SALVANDO RESULTADOS ({n_rounds} ROUNDS x 10-FOLD CROSS-VALIDATION)")
print(f"{'='*80}")

if len(all_results) == 0:
    print("\nAVISO: Nenhum resultado foi gerado.")
else:
    df_all_results = pd.DataFrame(all_results)
    df_all_results.to_csv('./results/2-0_cv_3rounds_10fold_resultados_completos.csv', index=False)
    print("Resultados completos salvos em: ./results/2-0_cv_3rounds_10fold_resultados_completos.csv")
  # Médias por anomalia, classificador e round
    mean_by_round = df_all_results.groupby(['ANOMALIA', 'CLASSIFICADOR', 'ROUND'])[['PRECISAO', 'REVOGACAO', 'F1']].mean()
    mean_by_round.to_csv('./results/2-0_cv_3rounds_10fold_medias_por_round.csv')
    print("Médias por round salvos em: ./results/2-0_cv_3rounds_10fold_medias_por_round.csv")
  # Médias gerais por anomalia e classificador (agregando todos os rounds)
    mean_results = df_all_results.groupby(['ANOMALIA', 'CLASSIFICADOR'])[['PRECISAO', 'REVOGACAO', 'F1']].mean()
    mean_results.to_csv('./results/2-0_cv_3rounds_10fold_medias_por_anomalia.csv')
    print("Médias por anomalia salvos em: ./results/2-0_cv_3rounds_10fold_medias_por_anomalia.csv")
  # Médias gerais por classificador (todos os rounds e folds)
    overall_mean = df_all_results.groupby('CLASSIFICADOR')[['PRECISAO', 'REVOGACAO', 'F1']].mean()
    overall_mean.to_csv('./results/2-0_cv_3rounds_10fold_medias_gerais.csv')
    print("Médias gerais salvos em: ./results/2-0_cv_3rounds_10fold_medias_gerais.csv")
    print(f"\n{'='*80}")
    print("RESULTADOS FINAIS - Médias Gerais por Classificador (3 Rounds x 10-Fold CV)")
    print(f"{'='*80}")
    print(overall_mean.sort_values('F1', ascending=False))

    # Estatísticas por round
    print(f"\n{'='*80}")
    print("Médias por Round")
    print(f"{'='*80}")
    round_stats = df_all_results.groupby(['ROUND', 'CLASSIFICADOR'])[['F1']].mean().unstack()
    print(round_stats)

# Tempo total
print(f'\nTempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')
print("\n==============================")
print("TESTES ESTATÍSTICOS - CROSS VALIDATION")
print("==============================")

from scipy.stats import friedmanchisquare, wilcoxon

# -----------------------------------
# 1. Pivot: F1 por fold
# -----------------------------------
pivot = df_all_results.pivot_table(
    values="F1",
    index=["ANOMALIA","FOLD"],
    columns="CLASSIFICADOR"
).dropna()

print("\nShape matriz estatística:", pivot.shape)

# -----------------------------------
# 2. FRIEDMAN
# -----------------------------------
friedman_stat, friedman_p = friedmanchisquare(*[pivot[col] for col in pivot.columns])

print("\nFRIEDMAN RESULT")
print("stat =", friedman_stat)
print("p =", friedman_p)

# -----------------------------------
# 3. WILCOXON pareado
# (LOF vs outros)
# -----------------------------------
if "Local Outlier Factor" in pivot.columns:
    base = pivot["Local Outlier Factor"]

    print("\nWILCOXON (vs LOF)")
    for col in pivot.columns:
        if col == "Local Outlier Factor":
            continue
        stat, p = wilcoxon(base, pivot[col])
        print(f"LOF vs {col}: p = {p}")


# Tempo total
print(f'\nTempo total de execução (hh:mm:ss.ms): {datetime.now() - start_time}')


Processando anomalia: Aumento Abrupto de BSW (código 1)

Total de instâncias encontradas: 5
  Carregando instância 1/5: WELL-00001_20140124213136.csv
    -> Ignorada (período normal insuficiente: 959)
  Carregando instância 2/5: WELL-00002_20140126200050.csv
    -> Ignorada (período normal insuficiente: 1138)
  Carregando instância 3/5: WELL-00006_20170801063614.csv
  Carregando instância 4/5: WELL-00006_20170802123000.csv
  Carregando instância 5/5: WELL-00006_20180618060245.csv

Instâncias utilizadas: 3
Instâncias ignoradas: 2
AVISO: Número insuficiente de instâncias (3) para CV robusta
Recomendado mínimo: 10. Pulando esta anomalia.


Processando anomalia: Fechamento Espúrio de DHSV (código 2)

Total de instâncias encontradas: 22
  Carregando instância 1/22: WELL-00002_20131104014101.csv
  Carregando instância 2/22: WELL-00003_20141122214325.csv
  Carregando instância 3/22: WELL-00003_20170728150240.csv
  Carregando instância 4/22: WELL-00003_20180206182917.csv
    -> Ignorada (perí